# Future Scope:
### 1. Use pyspark to create pipeline
### 2. Add testing to avoid corruption of data

In [8]:
!unzip -q /content/drive/MyDrive/Copy\ of\ data.zip

replace data/VisitorLogsData.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/userTable.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [2]:
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from datetime import timedelta

In [5]:
import datetime

In [6]:
def load_data(visfile,usrfile,dat):
  vis = pd.read_csv(visfile)
  usr = pd.read_csv(usrfile)
  return vis,usr

In [7]:
vis,usr = load_data('/content/data/VisitorLogsData.csv','/content/data/userTable.csv',12)

In [ ]:
class Pipe:
  def __init__(self,visdf,usrdf,day,month,year):
    self.vis = visdf
    self.usr = usrdf
    self.day = int(day)
    self.month = int(month)
    self.year = int(year)
    self.sub = pd.DataFrame()

  def clean_data(self):
    print('Cleaning Data.....')
    self.vis.dropna(subset = ['UserID'],inplace = True)
    self.vis['Activity'] = self.vis['Activity'].apply(lambda x: x.capitalize() if not pd.isnull(x) else x)
    self.vis['OS'] = self.vis['OS'].apply(lambda x: x.capitalize() if not pd.isnull(x) else x)
    self.vis['ProductID'] = self.vis['ProductID'].apply(lambda x: x.capitalize() if not pd.isnull(x) else x)
    self.vis = pd.merge(self.vis,self.usr,how = 'left',on='UserID')
    self.vis['Activity'] = self.vis['Activity'].bfill()
    self.vis.sort_values(by = 'UserID',inplace = True)
    self.sub['UserID'] = self.vis['UserID'].unique()

  def check_date(self):
    print('Cleaning Date.....')
    self.df2 = self.vis[~self.vis['VisitDateTime'].isnull()]
    self.df2['VisitDateTime'] = self.df2['VisitDateTime'].apply(lambda x: pd.to_datetime(int(x)//10**9,unit = 's',errors = 'ignore') if '-' not in x else pd.to_datetime(x,format = '%Y-%m-%d %H:%M:%S'))
    for x in self.df2.index:
      self.vis.loc[x,'VisitDateTime'] = self.df2.loc[x,'VisitDateTime']
    self.vis['VisitDateTime']=pd.to_datetime(self.vis['VisitDateTime'], format='%Y%m%d%H%M%S%f')
  def mergedf(self,df):
    self.sub = pd.merge(self.sub,df,how = 'left',on='UserID')

  def most_rec_viewed(self):
    print('Fetching Most Recent Viewed Products.....')
    self.df4 = self.vis[~self.vis['VisitDateTime'].isna()]
    self.df4 = self.df4[~self.df4['ProductID'].isna()]
    self.most_rec_view = self.df4[self.df4.groupby('UserID')['VisitDateTime'].transform(max)==self.df4['VisitDateTime']]
    self.most_rec_view.drop_duplicates(['ProductID','UserID'],inplace = True)
    self.most_rec_view = self.most_rec_view[['UserID','ProductID']]
    self.most_rec_view.rename(columns = {'ProductID':'Recently_Viewed_Product'},inplace = True)
    self.mergedf(self.most_rec_view)
    self.sub['Recently_Viewed_Product'].fillna('Product101',inplace = True)

  def most_active_os(self):
    print('Fetching Most Active OS.....')
    self.sub['Most_Active_OS'] = self.vis.groupby('UserID')['OS'].agg(pd.Series.mode).values

  def product_views(self):
    print('Fetching Product Views.....')
    self.df3 = self.vis[~self.vis['ProductID'].isna()]
    self.df3 = self.df3[~self.df3['VisitDateTime'].isna()]
    self.df3 = self.df3[self.df3['VisitDateTime']>=self.df3['VisitDateTime'].max()-timedelta(days=15)]
    self.num_prod_views = self.df3.groupby('UserID')['ProductID'].nunique()
    self.num_prod_views = pd.DataFrame(self.num_prod_views)
    self.num_prod_views.rename(columns={'ProductID':'No_Of_Products_Viewed_15_Days'},inplace = True)
    self.num_prod_views.reset_index(inplace = True)
    self.mergedf(self.num_prod_views)
    self.sub['No_Of_Products_Viewed_15_Days'].fillna('0',inplace = True)

  def pageloads_and_clicks(self):
    print('Fetching Pageloads and Clicks.....')
    self.df5 = self.vis[~self.vis['VisitDateTime'].isna()]
    self.df5.dropna(subset = ['Activity'],inplace = True)
    self.df5 = self.df5[self.df5['VisitDateTime']>= self.df5['VisitDateTime'].max()-timedelta(days=7)]
    self.pageloads = pd.DataFrame(self.df5[self.df5['Activity']=='Pageload'].groupby('UserID')['Activity'].count())
    self.pageloads.rename(columns = {'Activity':'Pageloads_last_7_days'},inplace = True)
    self.clicks = pd.DataFrame(self.df5[self.df5['Activity']=='Click'].groupby('UserID')['Activity'].count())
    self.clicks.rename(columns = {'Activity':'Clicks_last_7_days'},inplace = True)
    self.pageloads.reset_index(inplace = True)
    self.clicks.reset_index(inplace = True)
    self.mergedf(self.pageloads)
    self.mergedf(self.clicks)
    self.sub['Pageloads_last_7_days'].fillna(0,inplace = True)
    self.sub['Clicks_last_7_days'].fillna(0,inplace = True)

  def most_viewed(self):
    print('Fetching Most Viewed Product.....')
    self.df6 = self.vis[~self.vis['VisitDateTime'].isna()]
    self.last_15_days = self.df6[self.df6['VisitDateTime'] >= '2018-05-13']
    self.last_15_days = self.last_15_days[~self.last_15_days['ProductID'].isna()]
    self.last_15_days = self.last_15_days.groupby('UserID')['ProductID'].apply(lambda x: x.value_counts().index[0])
    self.last_15_days = pd.DataFrame(self.last_15_days)
    self.last_15_days.reset_index(inplace = True)
    self.last_15_days.rename(columns = {'ProductID':'Most_Viewed_product_15_Days'},inplace = True)
    self.mergedf(self.last_15_days)
    self.sub['Most_Viewed_product_15_Days'].fillna('Product101',inplace = True)

  def no_of_days_visited(self):
    print('Fetching No. of Days Visited.....')
    self.feature = self.vis[self.vis['VisitDateTime'] >= '2018-05-21']
    self.feature = self.feature[~self.feature['Activity'].isna()]
    self.feature["Date"] = self.feature['VisitDateTime'].dt.date
    self.No_of_day_Visited_7_days = self.feature.groupby('UserID')['Date'].nunique().reset_index(name='No_of_days_Visited_7_Days')
    self.mergedf(self.No_of_day_Visited_7_days)
    self.sub['No_of_days_Visited_7_Days'].fillna(0,inplace = True)

  def user_vintage(self):
    print('Fetching User Vintage.....')
    self.vis['Signup Date'] = pd.to_datetime(self.vis['Signup Date'], format='%Y-%m-%d %H:%M:%S.%f')
    self.df8 = self.vis[['Signup Date','UserID']]
    self.df8['Signup Date Filtered'] = self.df8['Signup Date'].dt.date
    self.df8['User_Vintage'] = datetime.date(year = self.year,month = self.month,day = self.day)-self.df8['Signup Date Filtered']
    self.df8['User_Vintage'] = self.df8['User_Vintage'].dt.days.astype(int)
    self.uv = self.df8.groupby('UserID')['User_Vintage'].mean()
    self.uv = pd.DataFrame(self.uv,columns = ['User_Vintage']).reset_index()
    self.mergedf(self.uv)

  def create_submission(self):
    self.clean_data()
    self.check_date()
    self.most_rec_viewed()
    self.most_active_os()
    self.product_views()
    self.pageloads_and_clicks()
    self.most_viewed()
    self.no_of_days_visited()
    self.user_vintage()
    self.sub['No_of_days_Visited_7_Days'] = self.sub['No_of_days_Visited_7_Days'].astype(int)
    self.sub['Pageloads_last_7_days'] = self.sub['Pageloads_last_7_days'].astype(int)
    self.sub['Clicks_last_7_days'] = self.sub['Clicks_last_7_days'].astype(int) 
    self.sub = self.sub[['UserID','No_of_days_Visited_7_Days','No_Of_Products_Viewed_15_Days','User_Vintage','Most_Viewed_product_15_Days','Most_Active_OS','Recently_Viewed_Product','Pageloads_last_7_days','Clicks_last_7_days']]
    self.sub.to_csv('Submission.csv',index = False)

In [ ]:
obj = Pipe(vis,usr,28,5,2018)

In [ ]:
obj.create_submission()

Cleaning Data.....
Cleaning Date.....
Fetching Most Recent Viewed Products.....
Fetching Most Active OS.....
Fetching Product Views.....
Fetching Pageloads and Clicks.....
Fetching Most Viewed Product.....
Fetching No. of Days Visited.....
Fetching User Vintage.....
